In [1]:
import pandas as pd
from espn_api.football import League

# Initialize the league
pgb_league = League(league_id=1328496691, year=2024, espn_s2='AEBYECasftI1k7UT5d4A7FbT1EfGRH660hjw%2FclbTqlKTY1a7KlqHIhaj4%2BoPD82tbfS8uOCS0nq%2B9G8wgnOuztJOi%2BCRcAIfue1xsW3vQUYDLJ8TlX9s5ieZPAcI3MvgLzDfzRkT0QWjEbq2MRZrzEnv2zM8jY3MepkKDtXWwE4aauJTL4d4Tq4Zeljxgsz7%2BH4KYkVBSkczEsQPUQldvmdNQmkGWU2E8nQwXgDVAawjYObLsqqO93cuJiu2qlbPsrR7b%2FfEVNn7b9%2FFRMyvmIG8ZUKvUFAe6iINuUrUd0779KsdgCtBbg6210B4W%2FQjgQ%3D', swid='{2C7AFCC4-F961-4FAB-B021-DD728F1DC1E9}')

# Retrieve all free agents
free_agents = pgb_league.free_agents()

# Define injury status values
injury_status_values = {
    'HEALTHY': 1.0,
    'DOUBTFUL': 0.75,
    'QUESTIONABLE': 0.5,
    'INJURY_RESERVE': 0.5,
    'OUT': 0.0
}

# Extract relevant stats for each player
free_agents_data = []
for player in free_agents:
    injury_status = tuple(player.injuryStatus) if isinstance(player.injuryStatus, list) else player.injuryStatus
    injury_value = injury_status_values.get(injury_status, 1.0)

    player_data = {
        'Name': player.name,
        'Points': player.points,
        'Projected': player.projected_points,
        'Position': player.position,
        'Team': player.proTeam,
        'Injury Status': injury_status,
        'Eligible Slots': player.eligibleSlots,
        'Acquisition Type': player.acquisitionType,
        'Injury Value': injury_value
    }
    free_agents_data.append(player_data)

# Create a pandas DataFrame from the extracted data
free_agents_df = pd.DataFrame(free_agents_data)

# Calculate the Value Ratio
free_agents_df['Value Ratio'] = free_agents_df['Projected'] / free_agents_df['Points']

# Add a new column for the ratio of eligible positions to projected points, factoring in injury status and value ratio
free_agents_df['PPR'] = (free_agents_df['Eligible Slots'].apply(lambda x: len(x)) / free_agents_df['Projected']) * free_agents_df['Injury Value'] * free_agents_df['Value Ratio']

# Sort the DataFrame by the new ratio column in descending order
sorted_free_agents_df = free_agents_df.sort_values(by='PPR', ascending=False)

# Display the sorted DataFrame
print(sorted_free_agents_df)

                  Name  Points  Projected Position  Team   Injury Status  \
29       Juwan Johnson    0.00       7.14       TE    NO          ACTIVE   
19        Mike Gesicki    0.00       7.22       TE   CIN          ACTIVE   
34     Brenton Strange    0.00       7.93       TE   JAX          ACTIVE   
16         Blake Corum    0.00       1.29       RB   LAR          ACTIVE   
38    Dontayvion Wicks    0.00       5.03       WR    GB          ACTIVE   
27           Zach Ertz    0.00       7.19       TE   WSH          ACTIVE   
42           Ray Davis    0.00       4.25       RB   BUF          ACTIVE   
18          Bills D/ST    0.00       5.69     D/ST   BUF              ()   
44         Tank Bigsby    0.00       4.80       RB   JAX    QUESTIONABLE   
33          Tyler Bass    0.00       8.17        K   BUF          ACTIVE   
31      Dustin Hopkins    0.00       8.31        K   CLE          ACTIVE   
2          Tee Higgins    0.00       9.55       WR   CIN          ACTIVE   
37      Khal

In [2]:
# Save the DataFrame to a CSV file
sorted_free_agents_df.to_csv('free_agents_2.csv', index=False)